In [ ]:
# default_exp utils

# Utils

> This file holds function to load the data and create folds for cross-validation.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai.vision.all import *
from sklearn.model_selection import StratifiedKFold

## Load Data

In [ ]:
#export
def load_data(with_folds=True) -> (Path, pd.DataFrame):
    path = Path("/home/jupyter/kaggle/plant-pathology/data/plant-pathology-2020/")
    path_train = path/('train_folds.csv' if with_folds else 'train.csv')
    df_train = pd.read_csv(path_train)
    return path, df_train

In [ ]:
path, df = load_data()
df.head(), path.ls()

## Create Folds

In [ ]:
#export
def create_folds(path: Path, df: pd.DataFrame, prn_stats: bool = False) -> pd.DataFrame:
    df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)
    lbls = df.apply(lambda r: df.columns[r==1].item(), axis=1)
    
    # Create 5 folds
    kf = StratifiedKFold(n_splits=5)
    for fold, (train_idxs, val_idxs) in enumerate(kf.split(df, lbls.values)):
        print(f"Fold {fold}: {len(train_idxs)/len(df)}, {len(val_idxs)/len(df)}")
        df.loc[val_idxs, "fold"] = fold
        
    if prn_stats: print(df.groupby("fold").describe())
    
    # Save to file
    df.to_csv(path/"train_folds.csv", index=False)
    return df

In [ ]:
path, df = load_data(False)
create_folds(path, df).head()

## Print Command to Submit to Kaggle

In [ ]:
def kaggle_submit_command() -> str:
    print("!kaggle competitions submit -c plant-pathology-2020-fgvc7 -f {save_path} -m 'hi'")

In [ ]:
kaggle_submit_command()

!kaggle competitions submit -c plant-pathology-2020-fgvc7 -f {save_path} -m 'hi'
